<a href="https://colab.research.google.com/github/timaZhuk/ml-native-algorithms/blob/main/LLM2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM from scratch (not Finished)

In [ ]:
%pip install pylzma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 32.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylzma: filename=pylzma-0.5.0-cp310-cp310-linux_x86_64.whl size=222322 sha256=a86c8e361bb52dddd8eb7e4e874178504633b6b2ab425c64496c941310009f3f
  Stored in directory: /root/.cache/pip/wheels/74/c9/02/91112815e838f544c1d46fda071241e454694579d022751d2b
Successfully built pylzma


## Read the file

In [ ]:
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cpu


In [ ]:
# https://www.gutenberg.org/cache/epub/22566/pg22566.txt

with open('/content/drive/MyDrive/wizard_of_oz.txt', 'r', encoding='utf-8') as f:
  text = f.read()
chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)
print(len(text))

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']
252022


In [ ]:
print(text[:150])

﻿The Project Gutenberg eBook of Dorothy and the Wizard in Oz
    
This ebook is for the use of anyone anywhere in the United States and
most other par


# Number of individual charachters in the text

In [ ]:
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [ ]:
# encode char to integer
string_to_int = {ch:i for i, ch in enumerate(chars)}
# decode int to char
int_to_string  = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [ ]:
# Encode 'hello' to array of integers
print(encode('hello'))

[65, 62, 69, 69, 72]


In [ ]:
# decode [65, 62, 69, 69, 72] array to string
print(decode([65, 62, 69, 69, 72]))

hello


In [ ]:
# convert text to pytorch tensor
data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([91, 48, 65, 62,  1, 44, 75, 72, 67, 62, 60, 77,  1, 35, 78, 77, 62, 71,
        59, 62, 75, 64,  1, 62, 30, 72, 72, 68,  1, 72, 63,  1, 32, 72, 75, 72,
        77, 65, 82,  1, 58, 71, 61,  1, 77, 65, 62,  1, 51, 66, 83, 58, 75, 61,
         1, 66, 71,  1, 43, 83,  0,  1,  1,  1,  1,  0, 48, 65, 66, 76,  1, 62,
        59, 72, 72, 68,  1, 66, 76,  1, 63, 72, 75,  1, 77, 65, 62,  1, 78, 76,
        62,  1, 72, 63,  1, 58, 71, 82, 72, 71])


In [ ]:
# split data into train and validation
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
# bigram models shifting training data
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print('when input is', context, 'target is', target)

when input is tensor([91]) target is tensor(48)
when input is tensor([91, 48]) target is tensor(65)
when input is tensor([91, 48, 65]) target is tensor(62)
when input is tensor([91, 48, 65, 62]) target is tensor(1)
when input is tensor([91, 48, 65, 62,  1]) target is tensor(44)
when input is tensor([91, 48, 65, 62,  1, 44]) target is tensor(75)
when input is tensor([91, 48, 65, 62,  1, 44, 75]) target is tensor(72)
when input is tensor([91, 48, 65, 62,  1, 44, 75, 72]) target is tensor(67)


# Intro to PyTorch

In [ ]:
import torch

In [ ]:
randint = torch.randint(-100, 100, (6,))
randint

tensor([ 93, -86,  16,  19, -15,  66])

In [ ]:
tensor = torch.tensor([
    [0.1, 1.2],
    [2.2, 3.1],
    [4.9, 5.2]
])
tensor

tensor([[0.1000, 1.2000],
        [2.2000, 3.1000],
        [4.9000, 5.2000]])

In [ ]:
# create tensor of zeros
zeros = torch.zeros(2, 3)
zeros

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [ ]:
ones = torch.ones(3, 4)
ones

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [ ]:
input = torch.empty(2, 3)
input

tensor([[2.1707e-18, 7.0952e+22, 1.7748e+28],
        [1.8176e+31, 7.2708e+31, 5.0778e+31]])

In [ ]:
arange = torch.arange(5)
arange

tensor([0, 1, 2, 3, 4])

In [ ]:
linespace = torch.linspace(3, 10, steps = 5)
linespace

tensor([ 3.0000,  4.7500,  6.5000,  8.2500, 10.0000])

In [ ]:
logspace = torch.logspace(start = -10, end = 10, steps = 5)

In [ ]:
eye = torch.eye(5)
eye

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])

In [ ]:
eye.shape

torch.Size([5, 5])

In [ ]:
a = torch.empty((2, 3), dtype=torch.int64)
empty_like = torch.empty_like(a)
empty_like

tensor([[94471603080000, 94471604008000,    -4294967295],
        [    8589934592,              3,            113]])

# Next section. Time perfomance

In [ ]:
import torch
import numpy as np
import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [ ]:
start_time = time.time()
# matrix operations here
zeros = torch.zeros(1, 1)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"{elapsed_time:.4f} ")

0.0258 


# Test the numpy and pyTorch

In [ ]:
torch_rand1 = torch.rand(10000, 10000).to(device)
torch_rand2 = torch.rand(10000, 10000).to(device)
#
np_rand1 = torch.rand(10000, 10000)
np_rand2 = torch.rand(10000, 10000)

start_time = time.time()

rand = (torch_rand1 @ torch_rand2)

end_time = time.time()

elapsed_time = end_time - start_time
print(f"{elapsed_time:.4f} ")


start_time = time.time()

rand = np.multiply(np_rand1, np_rand2)

end_time = time.time()

elapsed_time = end_time - start_time
print(f"{elapsed_time:.4f} ")


31.6590 
0.2011 


In [ ]:
# torch.stack, torch.multinominal, torch.tril
# torch.triu, input.T/input.transpose, nn.linear
# torch.cat, F.softmax (show all the examples of functions/)

# Define aprobability tensor
probabilities = torch.tensor([0.1, 0.9])
# 10% or 0.1 => 0, 90% or 0.9 => 1 each probability points to the index of the probability in the tensor
# Draw 5 samples from the multinominal distribution
samples = torch.multinomial(probabilities, num_samples = 10, replacement = True)
print(samples )

tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1])


In [ ]:
# concatenate the tensor together
tensor = torch.tensor([1, 2, 3, 4])
out = torch.cat((tensor, torch.tensor([5])), dim=0)
out

tensor([1, 2, 3, 4, 5])

In [ ]:
# lower trangle
out = torch.tril(torch.ones(5, 5))
out

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [ ]:
# upper trangle matrix
out = torch.triu(torch.ones(5, 5))
out

tensor([[1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.]])

In [ ]:
out = torch.zeros(5, 5).masked_fill(torch.tril(torch.ones(5,5))==0, float('-inf'))
out

tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])

In [ ]:
torch.exp(out)

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [ ]:
input = torch.zeros(2, 3, 4)
input

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [ ]:
out= input.transpose(0, 2)
out

tensor([[[0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.]]])

In [ ]:
out.shape

torch.Size([4, 3, 2])

In [ ]:
tensor1 = torch.tensor([1, 2, 3])
tensor2 = torch.tensor([4, 5, 6])
tensor3 = torch.tensor([7, 8, 9])

In [ ]:
# Stack the tensors along a new dimension
stacked_tensor =  torch.stack([tensor1, tensor2, tensor3])
stacked_tensor

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [ ]:
import torch.nn as nn
sample = torch.tensor([10.,10.,10.])
linear = nn.Linear(3, 3, bias=False)
print(linear(sample))

tensor([-1.0519,  7.5186, -6.0741], grad_fn=<SqueezeBackward4>)


In [ ]:
import torch.nn.functional as F

#Create a tensor
tensor1 = torch.tensor([1.0, 2.0, 3.0])

# Apply softmax using torch.nn.functional.softmax()
softmax_output = F.softmax(tensor1, dim=0)
print(softmax_output)

tensor([0.0900, 0.2447, 0.6652])


In [ ]:
vocab_size = 100000
embedding_dim = 100
embeding = nn.Embedding(vocab_size, embedding_dim)

#Create some input indices
input_indices = torch.LongTensor([1, 5, 3, 2])

#Apply the embedding layer
embedded_output = embeding(input_indices)

# the output will be a tensor of shape (4, 100) where 4 is the number of inputs
# end 100 is the dimensionality of the embeding vectors
print(embedded_output.shape)

torch.Size([4, 100])


In [ ]:
a = torch.tensor([
    [1,2],
    [3,4],
    [5,6]
    ])

b = torch.tensor([
    [7,8,9],
    [10,11,12]
])

print(a @ b)

tensor([[ 27,  30,  33],
        [ 61,  68,  75],
        [ 95, 106, 117]])


In [ ]:
# type int_64
int_64 = torch.randint(1, (3,2)).float()

# type float 32
float_32 = torch.rand(2,3)

result = torch.matmul(int_64, float_32)
print(result)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


# Batcj sizing recap model

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import torch.nn as nn
from torch.nn import functional as F
print(device)
block_size = 8
batch_size = 4

cpu


In [ ]:
# https://www.gutenberg.org/cache/epub/22566/pg22566.txt

with open('/content/drive/MyDrive/wizard_of_oz.txt', 'r', encoding='utf-8') as f:
  text = f.read()
chars = sorted(set(text)) # only unique characters in some order
print(chars)
vocabulary_size = len(chars)
print(len(text))
print(vocabulary_size)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']
252022
92


In [ ]:
# Dictionary where to each 'char' corresponed to integer(index)
string_to_int = {ch:i for i, ch in enumerate(chars)}

# Dictionary where to each 'integer' we get 'char'
int_to_string = {i:ch for i, ch in enumerate(chars)}

# encode string to numerical array
encode = lambda s: [string_to_int[c] for c in s]

# decode array of numbers to the string
decode = lambda l: ''.join([int_to_string[i] for i in l])

#example
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([91, 48, 65, 62,  1, 44, 75, 72, 67, 62, 60, 77,  1, 35, 78, 77, 62, 71,
        59, 62, 75, 64,  1, 62, 30, 72, 72, 68,  1, 72, 63,  1, 32, 72, 75, 72,
        77, 65, 82,  1, 58, 71, 61,  1, 77, 65, 62,  1, 51, 66, 83, 58, 75, 61,
         1, 66, 71,  1, 43, 83,  0,  1,  1,  1,  1,  0, 48, 65, 66, 76,  1, 62,
        59, 72, 72, 68,  1, 66, 76,  1, 63, 72, 75,  1, 77, 65, 62,  1, 78, 76,
        62,  1, 72, 63,  1, 58, 71, 82, 72, 71])


In [ ]:
# split data on tarining and validations sets
n = int(0.8*len(data))
#
train_data = data[:n]
val_data = data[n:]

In [ ]:
def get_batch(split):
  data = train_data if split == 'train' else val_data

  ix = torch.randint(len(data) - block_size, (batch_size,)) # make a tensor of randomly chosen [indeces from 'data']
  print(ix)
  # stack horisontally for x
  x = torch.stack([data[i:i+block_size] for i in ix])
  # for y shifted by 1
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

x, y = get_batch('train')
print('inputs, x:')
print(x.shape)
print(x)
print('trgets, y:')
print(y)

tensor([163794, 151814, 185723, 120998])
inputs, x:
torch.Size([4, 8])
tensor([[61,  1, 54, 62, 59, 12,  1, 58],
        [66, 71, 64,  1, 58,  1, 80, 66],
        [ 0,  0,  3, 36, 72, 80,  1, 80],
        [68, 82,  1, 76, 66, 61, 62, 76]])
trgets, y:
tensor([[ 1, 54, 62, 59, 12,  1, 58, 76],
        [71, 64,  1, 58,  1, 80, 66, 61],
        [ 0,  3, 36, 72, 80,  1, 80, 62],
        [82,  1, 76, 66, 61, 62, 76,  1]])


In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is: {context} target is {target}")

when input is: tensor([91]) target is 48
when input is: tensor([91, 48]) target is 65
when input is: tensor([91, 48, 65]) target is 62
when input is: tensor([91, 48, 65, 62]) target is 1
when input is: tensor([91, 48, 65, 62,  1]) target is 44
when input is: tensor([91, 48, 65, 62,  1, 44]) target is 75
when input is: tensor([91, 48, 65, 62,  1, 44, 75]) target is 72
when input is: tensor([91, 48, 65, 62,  1, 44, 75, 72]) target is 67


# Information:
### A bigram language model predicts the next word (or character) in a sequence based on the previous word (or character). It considers pairs of consecutive items (bigrams). For example, in the sentence "the cat sat," the bigrams are "the cat," "cat sat."
# Embedding Vectors (in this specific model):

### In this particular bigram model implementation, the embedding vectors have the same dimension as the vocabulary size. This means that the embedding vector for each token is a one-hot vector (or close to it after training) representing the probability distribution over the next word. For example, if after training the embedding vector for "cat" is [0.1, 0.6, 0.15, 0.05, 0.1], it means that the model predicts the next word to be "cat" with 60% probability, "the" with 10% probability, "sat" with 15% probability, and so on.
## * token_embedding_table
### The self.token_embedding_table is an nn.Embedding layer. You can think of it as a lookup table. It maps each token (represented by its numerical index) to a corresponding vector (the embedding vector).

### xample: Let's say your vocabulary has 5 words: "the," "cat," "sat," "on," "mat." You would assign each word a unique index (e.g., "the": 0, "cat": 1, "sat": 2, "on": 3, "mat": 4). The token_embedding_table would be a 5x5 matrix (in this case, because the embedding dimension is equal to the vocabulary size). Each row of this matrix is the embedding vector for the corresponding word.
## * Logits:
### In the context of classification (which language modeling essentially is), logits are the raw, unnormalized outputs of the model before applying a softmax function. In this bigram model, the logits directly represent the model's prediction of the next token.

### Example: If the input index is 1 (representing the word "cat"), self.token_embedding_table(1) will retrieve the embedding vector for "cat" (which is a vector of length 5 in this model). This vector is the logit and can be interpreted as the model's "preference" for each word in the vocabulary to be the next word.

# Why Use Embeddings?

### Semantic Similarity: Similar words have similar embedding vectors. This allows models to understand relationships between words.
### Dimensionality Reduction: Embeddings are much smaller than one-hot vectors, making computations more efficient.
Generalization: Embeddings allow models to generalize better to unseen words or contexts.
# Character-Level Embeddings:

### Character-level embeddings represent individual characters as vectors. This is particularly useful for:

### Handling Out-of-Vocabulary (OOV) words: If a word is not in the model's vocabulary, it can still be represented as a sequence of character embeddings.
### Morphological information: Character embeddings can capture prefixes, suffixes, and other morphological features of words.
### Languages with rich morphology: Languages with many word inflections benefit from character-level representations.

# Forward() method
### index: This is the input tensor containing the indices of the input tokens. It has a shape of (B, T), where:

## B is the batch size (the number of sequences processed in parallel).
## T is the sequence length (the number of tokens in each sequence).
## logits = ...: This line performs the embedding lookup. It takes the input indices and retrieves the corresponding embedding vectors from the token_embedding_table. The resulting logits tensor has a shape of (B, T, C), where C is the embedding dimension (or vocabulary size in a simplified bigram model).

## logits.view(B*T, C) reshapes the logits tensor. This is necessary to prepare the logits for the F.cross_entropy loss function.
# It combines the batch and time dimensions into a single dimension. The new shape is (B*T, C).

# Example (continuing from the previous example): logits was (2, 2, 3). After view(), it becomes (4, 3)

# targets = targets.view(B*T):

## targets: This tensor contains the indices of the target tokens (the next tokens in the sequence that the model should predict). It has a shape of (B, T).

## targets.view(B*T) reshapes the targets tensor to have a shape of (B*T). This is also necessary for the F.cross_entropy loss function. It flattens the target tensor into a 1D tensor.

## Example: If targets was [[2, 4], [0, 1]] (meaning the next tokens should be at indices 2, 4 in the first sequence and 0, 1 in the second sequence), after view(), it becomes [2, 4, 0, 1].
# 5. loss = F.cross_entropy(logits, targets):

# F.cross_entropy(logits, targets) calculates the cross-entropy loss between the predicted logits and the target indices.

# The cross_entropy loss function is commonly used for multi-class classification problems (like language modeling, where each word in the vocabulary is a class).

In [ ]:
class BigramLanguagemodel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # 5x5 matrix for example

  def forward(self, index, targets=None):
    logits = self.token_embedding_table(index)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      print(B, T, C)
      logits = logits.view(B*T, C)
      print("logits shape: ",logits.shape)
      targets = targets.view(B*T)
      print("targets shape: ",targets.shape)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, index, max_new_tokens):
    # index is (B, T) array of indecies in the current context
    for _ in range(max_new_tokens):
      # get the predictions
      logits, loss = self.forward(index)
      # focus only on the last time step
      logits = logits[:, -1 , :] # becomes (B, C)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim = -1) # (B, C)
      # sample from the distribution
      index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sample index to the running sequence
      index = torch.cat((index, index_next), dim=1) #(B, T+1)
    return index

In [ ]:
model = BigramLanguagemodel(vocabulary_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


$v5-
(XHT?‘O XorGNxQib(E—w]Qr,#”u ;’%d3C‘btKno sRC﻿f;"lXNu 30!C—Mrg“AoY;2Bwz3Y?iVHnPfOS9BSjM:PksGABp*4m90!™&%™PwCP8”TIS‘.BpjZ.—f™mjn6”﻿HT0uH—uHu‘5%81YaS96n.z’RPl
﻿
;PZf1;XIY.J,jdqe2z™FO*﻿—I.-XHZ;jZ1R)z—tsF5LHh''xJ2pHiYm6OuE?'?dfO[3—Eav’—“2-”.[!p!jT 8m.-]8TY0m"T™Ma56-—vpMN_JasT7Ph—#;QJ,d;xE#‘XaV59hSQ;UCQXegxrLN;NxRBSm0$6-'-&ZP,",1Q-L8%"M4BR1hg ‘k•wIoS#Q•tPu,W$[‘
P•FCUhLGf$6%eLm™”Z7fjd﻿)jci)jR”$x6WaK.Wv0?F1“vZa:OEC,vFy0vTkVPGTF1gqO')U]/
•7Yf]•trjF‘l—ZfFUa5WmMG5]0d﻿[8%LS﻿41.(:f!p﻿Lp_%TL[$.—rj-';


## a.view(x, y, z) is the key part. The view() function reshapes a tensor without copying the underlying data. It creates a new "view" of the same data in memory.
## In this specific case, a.view(x, y, z) is called with the same dimensions as the original tensor. This essentially does nothing. The shape remains (2, 3, 5).
# Important Considerations:

## view() creates a view, not a copy. Changes to the view will affect the original tensor, and vice versa.
## The total number of elements must remain the same when using view(). You can't change the number of elements.
## Sometimes, if the tensor is not contiguous in memory (due to previous operations like transpose() or permute()), you might need to call contiguous() before view() to ensure it works correctly.
##

In [ ]:
a = torch.rand(2, 3, 5)
print(a.shape)
x, y, z = a.shape
print(x, y, z)
a = a.view(x, y, z)
print(a.shape)

torch.Size([2, 3, 5])
2 3 5
torch.Size([2, 3, 5])
